In [10]:
# Book Analysis Script Runner
# This notebook runs the analyze_books.py script with proper arguments

import sys
import subprocess
from pathlib import Path

# Set up the arguments for the analysis script
root_dir = "."  # Current directory contains the biology_chapters, biology_pages, physics_chapters, physics_pages folders
output_dir = "./analysis_results"

# Create output directory if it doesn't exist
Path(output_dir).mkdir(exist_ok=True)

print(f"Root directory: {root_dir}")
print(f"Output directory: {output_dir}")
print(f"Available directories: {[d.name for d in Path('.').iterdir() if d.is_dir()]}")


Root directory: .
Output directory: ./analysis_results
Available directories: ['biology_pages', 'physics_chapters', 'analysis_results', 'biology_chapters', 'physics_pages']


In [11]:
# Run the book analysis script with proper arguments
# This will analyze both biology and physics books at chapter and page levels

cmd = [
    "python3", "analyze_books.py",
    "--root", root_dir,
    "--out", output_dir,
    "--levels", "chapters", "pages",
    "--modes", "intrabook", "interbook", 
    "--with-stopwords", "both",
    "--ngram-range", "1", "2",
    "--min-df", "2",
    "--workers", "4",
    "--topk", "50"
]

print("Running command:")
print(" ".join(cmd))
print("\n" + "="*50 + "\n")

# Execute the command
result = subprocess.run(cmd, capture_output=True, text=True)

print("STDOUT:")
print(result.stdout)
print("\nSTDERR:")
print(result.stderr)
print(f"\nReturn code: {result.returncode}")


Running command:
python3 analyze_books.py --root . --out ./analysis_results --levels chapters pages --modes intrabook interbook --with-stopwords both --ngram-range 1 2 --min-df 2 --workers 4 --topk 50


STDOUT:
Discovering documents...
Tokenizing (stopwords removed = True) ...


STDERR:

spaCy tokenize (stopwords=True):   0%|          | 0/3267 [00:01<?, ?it/s]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/concurrent/futures/process.py", line 254, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/Users/tanmayagarwal/Desktop/My_Computer/Columbia/Fall_2025/RA_LLM/drive-download-20251014T002431Z-1-001/analyze_books.py", line 108, in process_one
    nlp = spacy.load("en_core_web_sm", disable=[])  # load per-process
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/spacy/__init__.py", line 52, in load
    re

In [12]:
# Check the results if the analysis completed successfully
if result.returncode == 0:
    print("✅ Analysis completed successfully!")
    
    # List the generated files
    output_path = Path(output_dir)
    if output_path.exists():
        print(f"\nGenerated files in {output_dir}:")
        for file_path in sorted(output_path.rglob("*")):
            if file_path.is_file():
                print(f"  {file_path.relative_to(output_path)}")
        
        # Check if HTML report was generated
        html_report = output_path / "index.html"
        if html_report.exists():
            print(f"\n📊 HTML Report generated: {html_report}")
            print("Open this file in your browser to view the analysis results!")
    else:
        print(f"❌ Output directory {output_dir} was not created")
else:
    print("❌ Analysis failed. Check the error messages above.")


❌ Analysis failed. Check the error messages above.


In [13]:
#!/usr/bin/env python3
import argparse, os, re, json
from pathlib import Path
from functools import partial
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from concurrent.futures import ProcessPoolExecutor, as_completed
from jinja2 import Template

In [14]:
MD_PATTERN = re.compile(r"[#>\*\_\-\[\]\(\)\!\`\~]")

def read_markdown_lower(path: Path) -> str:
    # read & lowercase; strip obvious md syntax cheaply (we still tokenize w/ spaCy)
    text = path.read_text(encoding="utf-8", errors="ignore").lower()
    return MD_PATTERN.sub(" ", text)

def tokenize_doc(nlp, text, remove_stop=False, lemmatize=True):
    # fast tokenizer-only pipeline; disable heavy components
    with nlp.select_pipes(enable=["tok2vec","tagger","lemmatizer"] if lemmatize else []):
        doc = nlp(text)
    toks = []
    for t in doc:
        if not t.is_alpha:
            continue
        if remove_stop and (t.is_stop or t.text in STOP_WORDS):
            continue
        toks.append(t.lemma_ if (lemmatize and t.lemma_ != "-PRON-") else t.text)
    return toks

def type_token_ratio(tokens):
    return (len(set(tokens)) / max(1, len(tokens)))

def jaccard_set(a: set, b: set) -> float:
    if not a and not b: return 0.0
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union else 0.0

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def save_heatmap(matrix, labels, title, out_png):
    plt.figure(figsize=(max(6, len(labels)*0.22), max(5, len(labels)*0.22)))
    plt.imshow(matrix, aspect='auto', interpolation='nearest')
    plt.colorbar()
    plt.xticks(range(len(labels)), labels, rotation=90)
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    plt.close()

def write_csv(df, path):
    df.to_csv(path, index=True)


Discovery

In [15]:
def discover_docs(root: Path):
    """
    Returns list of dicts:
      { 'book': 'physics', 'level': 'chapters'|'pages', 'id': 'filename_no_ext', 'path': Path }
    """
    out = []
    for sub in sorted(root.iterdir()):
        if not sub.is_dir(): continue
        name = sub.name
        if name.endswith("_chapters"):
            book = name.replace("_chapters","")
            level = "chapters"
        elif name.endswith("_pages"):
            book = name.replace("_pages","")
            level = "pages"
        else:
            continue
        for md in sorted(sub.glob("*.md")):
            out.append({
                "book": book,
                "level": level,
                "id": md.stem,
                "path": md
            })
    return out


Processing

In [16]:
def process_one(nlp_bytes, item, remove_stop, lemmatize):
    # to make spaCy model picklable across processes
    nlp = spacy.blank("en")
    nlp = spacy.load("en_core_web_sm", disable=[])  # load per-process
    text = read_markdown_lower(item["path"])
    tokens = tokenize_doc(nlp, text, remove_stop=remove_stop, lemmatize=lemmatize)
    cnt = Counter(tokens)
    return {
        "book": item["book"],
        "level": item["level"],
        "id": item["id"],
        "path": str(item["path"]),
        "n_tokens": len(tokens),
        "n_types": len(cnt),
        "ttr": type_token_ratio(tokens),
        "top_tokens": json.dumps(cnt.most_common(200)),
        "tokens_joined": " ".join(tokens),   # for TF-IDF
        "vocab_set": set(cnt.keys())
    }

def parallel_process(items, remove_stop, lemmatize, workers):
    # nlp bytes unused but kept for structure
    results = []
    with ProcessPoolExecutor(max_workers=workers) as ex:
        futs = []
        for it in items:
            futs.append(ex.submit(process_one, None, it, remove_stop, lemmatize))
        for f in tqdm(as_completed(futs), total=len(futs), desc=f"spaCy tokenize (stopwords={remove_stop})"):
            results.append(f.result())
    return results

Matrices

In [17]:
def build_tfidf_and_cosine(rows, ngram_range=(1,1), min_df=2):
    corpus = [r["tokens_joined"] for r in rows]
    vec = TfidfVectorizer(ngram_range=ngram_range, min_df=min_df)
    X = vec.fit_transform(corpus)  # docs x terms (sparse)
    cos = cosine_similarity(X, dense_output=False)  # sparse result OK
    return vec, X, cos

def build_jaccard_matrix(rows):
    n = len(rows)
    mats = np.zeros((n,n), dtype=np.float32)
    sets = [r["vocab_set"] for r in rows]
    for i in range(n):
        mats[i,i] = 1.0
        si = sets[i]
        for j in range(i+1, n):
            sj = sets[j]
            v = jaccard_set(si, sj)
            mats[i,j] = mats[j,i] = v
    return mats

def build_pearson_corr_from_tfidf(X):
    # X: docs x terms (sparse). Normalize column-wise? Pearson on rows requires dense-ish.
    # We’ll compute on a dense doc x term after clipping size if huge, else use sampling trick.
    # For large X, do correlation on top-k features by variance for tractability.
    from sklearn.feature_selection import VarianceThreshold
    if X.shape[1] > 50000:
        # keep features with highest variance (simple heuristic)
        variances = np.array((X.power(2)).mean(axis=0) - (X.mean(axis=0).A1**2)).ravel()
        idx = np.argsort(variances)[-50000:]
        Xr = X[:, idx].toarray()
    else:
        Xr = X.toarray()
    # docs x features -> correlation between docs
    # np.corrcoef expects variables in rows by default; use rowvar=True
    corr = np.corrcoef(Xr)
    # numerical issues -> clip
    corr = np.nan_to_num(corr, nan=0.0, posinf=1.0, neginf=-1.0)
    return corr

Slicers

In [18]:
def slice_rows(rows, by=("book","level"), where=None):
    # where is dict filter, e.g., {"book": "physics", "level":"chapters"}
    if where is None: return rows
    out = []
    for r in rows:
        ok = True
        for k,v in where.items():
            if r[k] != v: ok=False; break
        if ok: out.append(r)
    return out

def interbook_pairs(rows, level):
    # group by book, keep only specified level
    rows = [r for r in rows if r["level"] == level]
    return rows  # full set; interbook computed by not filtering by book

Reporting

In [19]:
REPORT_HTML = """<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
<title>Book Analysis Report</title>
<style>
body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; margin: 24px; }
h1, h2, h3 { margin: 12px 0; }
.grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(340px, 1fr)); gap: 16px; }
.card { border: 1px solid #ddd; border-radius: 10px; padding: 12px; }
small { color: #666; }
table { border-collapse: collapse; width: 100%; }
td, th { border: 1px solid #ddd; padding: 6px 8px; }
</style>
</head>
<body>
<h1>Comprehensive Book Analysis</h1>
<p><small>Generated at: {{ generated_at }}</small></p>

<h2>Per-document statistics</h2>
<div class="grid">
{% for sec in stats %}
  <div class="card">
    <h3>{{ sec.title }}</h3>
    <p><a href="{{ sec.csv }}">CSV</a></p>
  </div>
{% endfor %}
</div>

<h2>Matrices & Heatmaps</h2>
<div class="grid">
{% for m in matrices %}
  <div class="card">
    <h3>{{ m.title }}</h3>
    <p><a href="{{ m.csv }}">Matrix CSV</a></p>
    <img src="{{ m.png }}" style="max-width:100%; height:auto;" />
  </div>
{% endfor %}
</div>

<h2>Leaderboards (Top Overlaps)</h2>
<div class="grid">
{% for lb in leaderboards %}
  <div class="card">
    <h3>{{ lb.title }}</h3>
    <p><a href="{{ lb.csv }}">CSV</a></p>
  </div>
{% endfor %}
</div>

</body>
</html>
"""

def top_pairs_from_matrix(matrix, labels, k=50, diag_is_one=True):
    pairs = []
    n = len(labels)
    for i in range(n):
        for j in range(i+1, n):
            val = matrix[i,j]
            pairs.append((labels[i], labels[j], float(val)))
    pairs.sort(key=lambda x: x[2], reverse=True)
    return pairs[:k]


Finally Main

In [20]:
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--root", required=True, help="Root folder containing *_chapters/*_pages folders")
    ap.add_argument("--levels", nargs="+", default=["chapters","pages"], choices=["chapters","pages"])
    ap.add_argument("--modes", nargs="+", default=["intrabook","interbook"], choices=["intrabook","interbook"])
    ap.add_argument("--with-stopwords", default="both", choices=["yes","no","both"])
    ap.add_argument("--ngram-range", nargs=2, type=int, default=[1,1])
    ap.add_argument("--min-df", type=int, default=2)
    ap.add_argument("--workers", type=int, default=4)
    ap.add_argument("--topk", type=int, default=50, help="top frequent tokens per doc/book")
    ap.add_argument("--out", required=True)
    args = ap.parse_args()

    root = Path(args.root)
    out = Path(args.out)
    ensure_dir(out)
    ensure_dir(out / "png")
    ensure_dir(out / "csv")

    print("Discovering documents...")
    all_items = discover_docs(root)
    if not all_items:
        raise SystemExit("No markdown files found under the expected structure.")

    # filter by requested levels
    all_items = [it for it in all_items if it["level"] in set(args.levels)]

    # Process both stopword modes as requested
    sw_modes = ["no","yes"] if args.with_stopwords == "both" else [args.with_stopwords]

    report_stats = []
    report_mats = []
    report_lbs = []

    for sw in sw_modes:
        remove_stop = (sw == "no")  # "no stopwords" => remove_stop True
        # NOTE: naming clarity for files
        mode_tag = "nostop" if remove_stop else "withstop"

        print(f"Tokenizing (stopwords removed = {remove_stop}) ...")
        rows = parallel_process(all_items, remove_stop=remove_stop, lemmatize=True, workers=args.workers)

        # Per-document stats
        stats_df = pd.DataFrame([{
            "book": r["book"],
            "level": r["level"],
            "id": r["id"],
            "path": r["path"],
            "n_tokens": r["n_tokens"],
            "n_types": r["n_types"],
            "ttr": r["ttr"]
        } for r in rows]).sort_values(["book","level","id"])
        stats_csv = out / "csv" / f"doc_stats_{mode_tag}.csv"
        write_csv(stats_df, stats_csv)
        report_stats.append({"title": f"Per-doc stats ({mode_tag})", "csv": str(stats_csv.relative_to(out))})

        # Also aggregate top tokens per book
        agg = defaultdict(Counter)
        for r in rows:
            agg[(r["book"], r["level"])].update(json.loads(r["top_tokens"]))
        agg_rows = []
        for (book, level), cnt in agg.items():
            for tok, c in cnt.most_common(args.topk):
                agg_rows.append({"book":book, "level":level, "token":tok, "count":c})
        top_csv = out / "csv" / f"top_tokens_{mode_tag}.csv"
        write_csv(pd.DataFrame(agg_rows), top_csv)
        report_stats.append({"title": f"Top-{args.topk} tokens by book/level ({mode_tag})", "csv": str(top_csv.relative_to(out))})

        # Labels
        labels = [f"{r['book']}::{r['level']}::{r['id']}" for r in rows]

        # Build TF-IDF + Cosine & Pearson
        vec, X, cos = build_tfidf_and_cosine(rows, ngram_range=tuple(args.ngram_range), min_df=args.min_df)
        cos = cos.A if hasattr(cos, "A") else np.array(cos)  # dense for saving/plotting
        pearson = build_pearson_corr_from_tfidf(X)

        # Build Jaccard
        jac = build_jaccard_matrix(rows)

        # Save full matrices
        def save_matrix(mat, name, title):
            df = pd.DataFrame(mat, index=labels, columns=labels)
            csv_path = out / "csv" / f"{name}_{mode_tag}.csv"
            write_csv(df, csv_path)
            png_path = out / "png" / f"{name}_{mode_tag}.png"
            save_heatmap(mat, labels, title, png_path)
            report_mats.append({"title": f"{title} ({mode_tag})",
                                "csv": str(csv_path.relative_to(out)),
                                "png": str(png_path.relative_to(out))})

        save_matrix(jac, "jaccard_all", "Jaccard Overlap (All Docs)")
        save_matrix(cos, "cosine_all", "Cosine Similarity (TF-IDF, All Docs)")
        save_matrix(pearson, "pearson_all", "Pearson Correlation (All Docs)")

        # Intra / Inter slicing per requested modes and per level
        for level in set([r["level"] for r in rows]):
            idx_level = [i for i,r in enumerate(rows) if r["level"]==level]
            def sub(mat):
                idx = np.ix_(idx_level, idx_level)
                return mat[idx]

            if "intrabook" in args.modes:
                # per book
                for book in sorted(set(r["book"] for r in rows)):
                    idx = [i for i,r in enumerate(rows) if r["level"]==level and r["book"]==book]
                    if len(idx) < 2: continue
                    labels_sub = [labels[i] for i in idx]
                    for (mat, name, title) in [(jac,"jaccard","Jaccard"),
                                               (cos,"cosine","Cosine"),
                                               (pearson,"pearson","Pearson")]:
                        ms = mat[np.ix_(idx, idx)]
                        df = pd.DataFrame(ms, index=labels_sub, columns=labels_sub)
                        csv_path = out / "csv" / f"{name}_intrabook_{book}_{level}_{mode_tag}.csv"
                        write_csv(df, csv_path)
                        png_path = out / "png" / f"{name}_intrabook_{book}_{level}_{mode_tag}.png"
                        save_heatmap(ms, labels_sub, f"{title} ({book} / {level})", png_path)
                        report_mats.append({"title": f"{title} – Intrabook {book} ({level}, {mode_tag})",
                                            "csv": str(csv_path.relative_to(out)),
                                            "png": str(png_path.relative_to(out))})

                    # leaderboards (top pairs) for this intrabook slice using Jaccard
                    pairs = top_pairs_from_matrix(ms, labels_sub, k=50)
                    lb_df = pd.DataFrame(pairs, columns=["doc_a","doc_b","score"])
                    lb_path = out / "csv" / f"leaderboard_intrabook_{book}_{level}_{mode_tag}.csv"
                    write_csv(lb_df, lb_path)
                    report_lbs.append({"title": f"Top Overlaps – Intrabook {book} ({level}, {mode_tag})",
                                       "csv": str(lb_path.relative_to(out))})

            if "interbook" in args.modes:
                # same level, across books (all docs of that level together)
                labels_lvl = [labels[i] for i in idx_level]
                for (mat, name, title) in [(jac,"jaccard","Jaccard"),
                                           (cos,"cosine","Cosine"),
                                           (pearson,"pearson","Pearson")]:
                    ms = sub(mat)
                    df = pd.DataFrame(ms, index=labels_lvl, columns=labels_lvl)
                    csv_path = out / "csv" / f"{name}_interbook_{level}_{mode_tag}.csv"
                    write_csv(df, csv_path)
                    png_path = out / "png" / f"{name}_interbook_{level}_{mode_tag}.png"
                    save_heatmap(ms, labels_lvl, f"{title} – Interbook ({level})", png_path)
                    report_mats.append({"title": f"{title} – Interbook ({level}, {mode_tag})",
                                        "csv": str(csv_path.relative_to(out)),
                                        "png": str(png_path.relative_to(out))})

                # interbook leaderboard (Jaccard)
                pairs = top_pairs_from_matrix(sub(jac), labels_lvl, k=50)
                lb_df = pd.DataFrame(pairs, columns=["doc_a","doc_b","score"])
                lb_path = out / "csv" / f"leaderboard_interbook_{level}_{mode_tag}.csv"
                write_csv(lb_df, lb_path)
                report_lbs.append({"title": f"Top Overlaps – Interbook ({level}, {mode_tag})",
                                   "csv": str(lb_path.relative_to(out))})

    # Render a simple index.html
    html = Template(REPORT_HTML).render(
        generated_at=pd.Timestamp.now().isoformat(timespec="seconds"),
        stats=report_stats, matrices=report_mats, leaderboards=report_lbs
    )
    (out / "index.html").write_text(html, encoding="utf-8")
    print(f"\n✅ Done. Open {out/'index.html'} in your browser.")

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] --root ROOT
                             [--levels {chapters,pages} [{chapters,pages} ...]]
                             [--modes {intrabook,interbook} [{intrabook,interbook} ...]]
                             [--with-stopwords {yes,no,both}]
                             [--ngram-range NGRAM_RANGE NGRAM_RANGE]
                             [--min-df MIN_DF] [--workers WORKERS]
                             [--topk TOPK] --out OUT
ipykernel_launcher.py: error: the following arguments are required: --root, --out


SystemExit: 2

/Users/tanmayagarwal/Library/Python/3.13/lib/python/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
